In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, IA3Config, TaskType
import mindspore
import datasets
from mindnlp.transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import numpy as np
from mindspore.dataset import GeneratorDataset

model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 8e-3
num_epochs = 3
batch_size = 8

/root/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.517 seconds.
Prefix dict has been built successfully.


In [2]:
# creating model
peft_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, feedforward_modules=[])

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)

800B [00:00, 243kB/s]                    
100%|██████████| 4.58G/4.58G [13:36<00:00, 6.02MB/s] 


In [3]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 282,624 || all params: 1,229,863,936 || trainable%: 0.022980103060766553


In [4]:
def process_dataset(source,   batch_size=32, shuffle=False):

    column_names = ['input_ids', 'attention_mask','labels','text_labels']
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms

    dataset = dataset.batch(batch_size)

    return dataset

class MSDataset:
    def __init__(self, filepath,tokenizer,max_length):
        self.path = filepath
        self.sentences = []
        self.labels = []
        self.text_labels = []
        self._load()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _load(self):
        label_mapping = {
            "negative": 0,
            "neutral": 1,
            "positive": 2
        }
        with open(self.path, encoding="iso-8859-1") as f:
            for line in f:
                sentence, label_text = line.strip().split("@")
                self.sentences.append(sentence)
                self.labels.append(label_mapping[label_text])
                self.text_labels.append(label_text)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        text_labels = self.text_labels[index]
        model_inputs = self.tokenizer(sentence, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="np")
        labels = self.tokenizer(text_labels, max_length=3, padding="max_length", truncation=True, return_tensors="np")
        labels = labels["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels,self.text_labels[index]

    def __len__(self):
        return len(self.sentences)
    
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
dataset = process_dataset(MSDataset("/tmp/code/dataset/data/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt",tokenizer,max_length),batch_size=batch_size)

train_dataset, eval_dataset = dataset.split([0.9, 0.1])



100%|██████████| 430/430 [00:00<00:00, 226kB/s]
100%|██████████| 4.11M/4.11M [00:01<00:00, 3.55MB/s]
100%|██████████| 74.0/74.0 [00:00<00:00, 22.5kB/s]


In [5]:
# optimizer and lr scheduler
optimizer = mindspore.nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr)


In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[   486,  23853,    339 ...      0,      0,      0]],
 [[   486,  80234,    339 ...      0,      0,      0]],
 [[   486,  13963,    305 ...      0,      0,      0]],
 ...
 [[111079,  15042,    259 ...      0,      0,      0]],
 [[   581,  14777,    259 ...      0,      0,      0]],
 [[  5126,    259,   2220 ...      0,      0,      0]]]), 'attention_mask': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 ...
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]],
 [[1, 1, 1 ... 0, 0, 0]]]), 'labels': Tensor(shape=[8, 1, 3], dtype=Int64, value=
[[[59006,     1,  -100]],
 [[59006,     1,  -100]],
 [[59006,     1,  -100]],
 ...
 [[59006,     1,  -100]],
 [[59006,     1,  -100]],
 [[59006,     1,  -100]]]), 'text_labels': Tensor(shape=[8], dtype=String, value= ['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral',
 'neutral', 'neutral'])}


In [7]:
# training and evaluation
from mindspore import Tensor

num_batches = len(train_dataset)
num_batches_eval = len(eval_dataset)
                       
def forward_fn(input_ids,attention_mask,labels ):
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        return output.loss, output.logits
    
grad_fn = mindspore.value_and_grad(
        forward_fn, None,optimizer.parameters, has_aux=True
    )
for epoch in range(num_epochs):
    model.set_train(True)
    total_loss, total_step = 0, 0
    correct = 0
    total = 0
    with tqdm(total=num_batches) as t:
        for step, (input_ids,attention_mask,labels,_) in enumerate(train_dataset):
            input_ids  = input_ids.squeeze(axis=1)
            labels  = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            (loss, logits), grad = grad_fn(input_ids,attention_mask,labels)
            optimizer(grad)
            total_loss += loss.asnumpy()
            total_step += 1
            curr_loss = total_loss / total_step
            t.set_postfix({'train-loss': f'{curr_loss:.2f}'})
            t.update(1)
    model.set_train(False)
    eval_loss = 0
    total_step = 0
    eval_preds = []
    text_labels = []
    with tqdm(total=num_batches_eval) as t:
        for step, (input_ids,attention_mask,labels,text) in enumerate(eval_dataset):
            input_ids = input_ids.squeeze(axis=1)
            labels = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            outputs = model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
            loss = outputs.loss
            eval_loss += loss.asnumpy()
            total_step += 1           
            eval_loss = total_loss / total_step
            eval_preds.extend(
                tokenizer.batch_decode(np.argmax(outputs.logits.asnumpy(), -1), skip_special_tokens=True)
            )
            text_str = str(text.asnumpy())
            text_str = text_str.replace("[", "").replace("]", "").replace(",", "").replace("'", "")
            labels = text_str.split(" ")
            text_labels.extend(labels)
            t.set_postfix({'eval-loss': f'{eval_loss:.2f}'})
            t.update(1)
    for pred, text_label in zip(eval_preds, text_labels):
        if pred.strip() == text_label.strip():
            correct += 1
        total += 1
    accuracy = correct / total * 100
    print(f"{accuracy=} % on the evaluation dataset")
    eval_epoch_loss = eval_loss / eval_dataset.get_dataset_size()
    eval_ppl = np.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / train_dataset.get_dataset_size()
    train_ppl = np.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 28/28 [00:09<00:00,  3.03it/s, eval-loss=2.31]


accuracy=92.85714285714286 % on the evaluation dataset
epoch=0: train_ppl=1.2880981384834842 train_epoch_loss=0.2531668192542651 eval_ppl=1.0858290532920414 eval_epoch_loss=0.08234379962989491


100%|██████████| 28/28 [00:09<00:00,  2.82it/s, eval-loss=0.60]


accuracy=97.32142857142857 % on the evaluation dataset
epoch=1: train_ppl=1.0683069944425718 train_epoch_loss=0.06607514721268386 eval_ppl=1.021723879588143 eval_epoch_loss=0.02149127874902345


100%|██████████| 28/28 [00:09<00:00,  2.87it/s, eval-loss=0.44]

accuracy=97.32142857142857 % on the evaluation dataset
epoch=2: train_ppl=1.0495465610914443 train_epoch_loss=0.0483582243172671 eval_ppl=1.01585310720886 eval_epoch_loss=0.015728759184825396


In [8]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [9]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


1.1M	bigscience/mt0-large_IA3_SEQ_2_SEQ_LM/adapter_model.ckpt


In [13]:
from mindnlp.peft import PeftModel, PeftConfig
import logging
logging.getLogger('mindspore').setLevel(logging.ERROR)
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

[WARNING] ME(747:140309892494272,MainProcess):2024-04-19-13:20:45.631.64 [mindspore/train/serialization.py:1378] For 'load_param_into_net', 558 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(747:140309892494272,MainProcess):2024-04-19-13:20:45.643.97 [mindspore/train/serialization.py:1383] base_model.shared.weight is not loaded.
[WARNING] ME(747:140309892494272,MainProcess):2024-04-19-13:20:45.651.84 [mindspore/train/serialization.py:1383] base_model.encoder.block.0.layer.0.SelfAttention.q.weight is not loaded.
[WARNING] ME(747:140309892494272,MainProcess):2024-04-19-13:20:45.658.98 [mindspore/train/serialization.py:1383] base_model.encoder.block.0.layer.0.SelfAttention.k.weight is not loaded.
[WARNING] ME(747:140309892494272,MainProcess):2024-04-19-13:20:45.666.25 [mindspore/train/serialization.py:1383] base_model.encoder.block.0.layer.0.Se

In [11]:
i = 13
def load_dataset(filepath):
    sentences = []
    labels = []
    with open(filepath, encoding="iso-8859-1") as f:
        for line in f:
            sentence, label = line.strip().split("@")
            sentences.append(sentence)
            labels.append(label)
    
    dataset = datasets.Dataset.from_dict({
        "sentence": sentences,
        "label": labels,
    })
    return dataset

dataset = load_dataset("/tmp/code/dataset/data/FinancialPhraseBank-v1.0/Sentences_AllAgree.txt")
dataset = dataset.train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="ms")
print(dataset["validation"][text_column][i])
print(inputs)

outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(outputs)
print(tokenizer.batch_decode(outputs ,skip_special_tokens=True))

/root/miniconda3/lib/python3.8/site-packages/mindnlp/transformers/generation/utils.py:1426: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


The total value of the contract is some EUR 8 million .
{'input_ids': Tensor(shape=[1, 14], dtype=Int64, value=
[[ 486, 2725, 9387 ...  259,  260,    1]]), 'attention_mask': Tensor(shape=[1, 14], dtype=Int64, value=
[[1, 1, 1 ... 1, 1, 1]])}
[[    0 59006     1]]
['neutral']
